In [1]:
import pandas as pd
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [7]:
def f(rank):
    return 1/np.sqrt(rank)

In [8]:
def ranking_calculation(df, f):
    
    n_players = len(df.Joueur.unique())

    ### Points de chaque match
    points = []
    for match in df.Match.unique():
        rank = list(df[df["Match"]==match]["Classement"])        
        sets = list(df[df["Match"]==match]["Scores"])
        for i in range(2):
            point = sets[i]*f(rank[1-i])
            points.append(point)                
        
    df["Points"] = points
    
    ### Calcul des stats de chaque joueur
    df_ranks = pd.DataFrame(columns={"Rang", "Joueur"})
    
    df_ranks["Joueur"] = df.Joueur.unique()
    played, wins, defeats, rate, points = [], [], [], [], []
    for player in df_ranks["Joueur"]:
        matchs_played = list(df[df["Joueur"]==player]["Match"])
        nb_played = df[df["Match"].isin(matchs_played)]["Scores"].sum()
        nb_wins = df[df["Joueur"]==player]["Scores"].sum()
        nb_defeats = nb_played - nb_wins
        
        played.append(nb_played)
        wins.append(nb_wins)
        defeats.append(nb_defeats)
        rate.append("{}%".format(int(nb_wins/nb_played*100)))
        points.append(round(df[df["Joueur"]==player]["Points"].sum(), 2))

    df_ranks["Sets joués"] = played
    df_ranks["Gagnés"] = wins
    df_ranks["Perdus"] = defeats
    df_ranks["Taux de succès"] = rate
    df_ranks["Points"] = points

    df_ranks.sort_values(by=["Points", "Taux de succès"], ascending=[False, False], inplace=True)
    
    df_ranks["Rang"] = range(1, n_players+1)
    
    return df_ranks

In [9]:
def ranking_squash(filename, f, calc_sets):
    
    df_matchs = pd.read_csv(filename, sep=';', decimal=',')
    
    if df_matchs.Classement.isna().any():
                
        df_previous_matchs = df_matchs.dropna().copy()

        df_previous_rank = ranking_calculation(df_previous_matchs, f)
        
        
        # Ajout des nouveaux joueurs
        count_new_players = len(df_matchs.Joueur.unique()) - len(df_previous_rank.Joueur.unique())
        
        if count_new_players != 0:
            print("There are {} new players".format(count_new_players))
            for player in df_matchs.Joueur.unique():
                if player not in df_previous_rank.Joueur.unique():
                    new_player = [player, len(df_previous_rank.Joueur.unique())+1] + [0]*(len(df_previous_rank.columns) - 2)
                    row = pd.Series(new_player, index=df_previous_rank.columns)
                    df_previous_rank = df_previous_rank.append(row, ignore_index=True)
                    
        # Remplissage avec les nouveaux classements
        for i in range(df_matchs.shape[0]):
            if np.isnan(df_matchs["Classement"][i]):
                player = df_matchs["Joueur"][i]
                previous_rank = int(df_previous_rank[df_previous_rank["Joueur"]==player]["Rang"])
                df_matchs["Classement"][i] = previous_rank

        # Export de l'historique mis à jour
        df_matchs.to_csv(filename, sep=';', decimal=',', columns=["Match", "Joueur", "Classement", "Scores"], index=False)
    
    # Classement final
    df_rank = ranking_calculation(df_matchs, f)
    
    return df_rank

In [10]:
rank = ranking_squash("Historique.csv", f, True)
rank

Joueur  Rang  Sets joués  Gagnés  Perdus Taux de succès  Points
3        KRa     1          74      66       8            89%   28.09
6     Mateos     2          65      25      40            38%   12.54
1      Colas     3          21      16       5            76%    8.43
0      Marin     4          35      14      21            40%    6.72
5    Gessien     5          23      12      11            52%    6.00
2         LL     6          38      13      25            34%    5.51
4     Sutter     7          31       8      23            25%    3.86
7        PLN     8          30       4      26            13%    2.50
8   Toujouse     9          11       6       5            54%    2.45
10     Alice    10           8       5       3            62%    1.58
9     Xavier    11           8       3       5            37%    0.90

In [11]:
df_points = pd.DataFrame(columns={"Joueur", "Points à gagner (sur un set)"})
df_points["Joueur"] = rank.Joueur
n_players = len(list(df_points["Joueur"]))
df_points["Points à gagner (sur un set)"] = [round(f(i),2) for i in range(1, n_players + 1)]

df_points

Points à gagner (sur un set)    Joueur
3                           1.00       KRa
6                           0.71    Mateos
1                           0.58     Colas
0                           0.50     Marin
5                           0.45   Gessien
2                           0.41        LL
4                           0.38    Sutter
7                           0.35       PLN
8                           0.33  Toujouse
10                          0.32     Alice
9                           0.30    Xavier